<h2> Assignment 5 Notebook </h2> 
<body>
<p>Full name : Kirthanaa Raghuraman</p>
<p>Wisc Id : kraghuraman</p>
<p>Test Setup details : </p>
<p>Jupyter verison : </p>
</body>

In [1]:
import pandas as pd
# numpy has a lots of useful math related modules 
import numpy as np
# Helpful function to display intermittent result 
from IPython.display import display


In [3]:
import plotly
api_key = 'x2VydZH1XsaHpEunc8ZG'
plotly.tools.set_credentials_file(username='kraghuraman', api_key=api_key)


<H4>Step 0 : Take a peek at the dataset</H4>

In [6]:
# Load the data and display only first five rows 
# NOTE replace the following value with the actual path to the csv file 
data_file = "AQI.csv" # e.g ~/cs564/p5/AQI.csv"
display(pd.read_csv(data_file, nrows=5).head())

,State Code,County Code,Latitude,Longitude,Date Local,AQI,Address,State Name,County Name,City Name,CBSA Name
0,55,59,42.504722,-87.8093,1997-04-28,82,"CHIWAUKEE PRAIRIE, 11838 FIRST COURT",Wisconsin,Kenosha,Pleasant Prairie,"Chicago-Naperville-Elgin, IL-IN-WI"
1,55,59,42.504722,-87.8093,1997-05-10,33,"CHIWAUKEE PRAIRIE, 11838 FIRST COURT",Wisconsin,Kenosha,Pleasant Prairie,"Chicago-Naperville-Elgin, IL-IN-WI"
2,55,59,42.504722,-87.8093,1997-05-16,25,"CHIWAUKEE PRAIRIE, 11838 FIRST COURT",Wisconsin,Kenosha,Pleasant Prairie,"Chicago-Naperville-Elgin, IL-IN-WI"
3,55,59,42.504722,-87.8093,1997-05-22,25,"CHIWAUKEE PRAIRIE, 11838 FIRST COURT",Wisconsin,Kenosha,Pleasant Prairie,"Chicago-Naperville-Elgin, IL-IN-WI"
4,55,59,42.504722,-87.8093,1997-05-28,46,"CHIWAUKEE PRAIRIE, 11838 FIRST COURT",Wisconsin,Kenosha,Pleasant Prairie,"Chicago-Naperville-Elgin, IL-IN-WI"


<H5>Step 1 : Load the dataset</H5>


In [8]:
from sqlalchemy import create_engine
import sqlite3
# create a database where we'll load the dataset from the csv file 
db_conn = create_engine('sqlite:///AirQualityIndex.db')

In [7]:
import string
# load in a batch of 5000 tuples. Modify this value to your needs
chunks = 5000
for data in pd.read_csv(data_file, chunksize=chunks,
 iterator=True, encoding='utf-8'):

 data = data.rename(columns={col: col.replace('-', ' ') for col in data.columns})
 data = data.rename(columns={col: col.strip() for col in data.columns})
 data = data.rename(columns={col: string.capwords(col) for col in data.columns})
 data = data.rename(columns={col: col.replace(' ', '') for col in data.columns})

 data.to_sql('data', db_conn, if_exists='append')

<H5>Step 2 : Data Exploration</H5>
<body>
<p> Find the average air quality index for each city recorded during the year 1997 </p>
</body>

In [4]:
avg_air_quality = pd.read_sql_query("""SELECT AVG(AQI) as avg_aqi, 
                             CityName as city FROM data
                             WHERE "DateLocal" LIKE "1997-%" 
                             GROUP BY city 
                             ORDER BY avg_aqi DESC""", db_conn)

# Display the result (Note - the result 'avg_air_quality' is an instance of 'pandas DataFrame')
display(avg_air_quality)

,avg_aqi,city
0,58.068182,Milwaukee
1,54.279070,Waukesha
2,50.731707,Pleasant Prairie


In [24]:
avg_aqi = pd.read_sql_query("""SELECT CityName, AVG(aqi) AS Avg_AQI 
                            FROM data 
                            WHERE CityName IN ('New York', 'Chicago', 'Boston', 'Los Angeles', 'San Francisco') 
                            AND DateLocal LIKE "2016-%" GROUP BY CityName ORDER BY Avg_AQI""", db_conn)
display(avg_aqi)

,CityName,Avg_AQI
0,Boston,26.766040
1,New York,28.603001
2,San Francisco,29.372881
3,Chicago,34.735294
4,Los Angeles,44.765000


In [26]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=avg_aqi['CityName'],
            y=avg_aqi['Avg_AQI'],
            marker=dict(
                color=['rgba(222,45,38,0.8)', 'rgba(222,45,38,0.8)',
               'rgba(222,45,38,0.8)', 'rgba(222,45,38,0.8)',
               'rgba(222,45,38,0.8)']),
    )]

layout = dict(title = 'Comparison of AQI of 5 cities in 2016',
              xaxis = dict(title = 'City'),
              yaxis = dict(title = 'Avg AQI'),
              )

fig = dict(data=data, layout=layout)


py.iplot(fig, filename='basic-bar')

In [23]:
SELECT CityName, AVG(aqi) AS Avg_AQI, SUBSTR(DateLocal,1,4) AS Year 
FROM data 
WHERE CityName IN ('New York', 'Chicago', 'Boston', 'Los Angeles', 'San Francisco') 
AND DateLocal Like '20__-%' 
GROUP BY CityName, SUBSTR(DateLocal,1,4);

SyntaxError: invalid syntax (<ipython-input-23-4fa92ab32c27>, line 1)

<H5>Step 3 : Data Visualization</H5>
<body>
<p>Visualize the data you received above as plots</p>
</body>

In [ ]:
Query 1 : Best month to visit new york. Line graph
Query 2 : Percentage change in air from year1 to year2. Plot on map.
Query 3 : Scatter plot. Lat vs Long. Agg by one year
Querr 4 : 

In [9]:
# Query 1 - Best time to visit new york

newyork_monthly_aqi = pd.read_sql_query("""SELECT Temp.Month AS Month, AVG(Temp.AQI) AS Avg_AQI FROM 
        (SELECT SUBSTR(DateLocal, 1,4) AS Year, SUBSTR(DateLocal, 6, 2) AS Month, SUBSTR(DateLocal, 9, 2) AS Date, AQI 
        FROM data 
        WHERE CityName = 'New York' AND Year LIKE '201_') AS Temp
        GROUP BY Temp.Month
        ORDER BY Temp.Month""", db_conn)

display(newyork_monthly_aqi)


,Month,Avg_AQI
0,01,42.767296
1,02,41.424701
2,03,31.981119
3,04,29.628388
4,05,36.145946
5,06,37.538117
6,07,43.086988
7,08,36.922611
8,09,31.870624
9,10,29.489765


In [10]:
import matplotlib.pyplot as plt
from pandas import DataFrame

month_list = list(newyork_monthly_aqi['Month'].values.flatten())
aqi_list = list(newyork_monthly_aqi['Avg_AQI'].values.flatten())
plt.title('Best time to visit New York')
plt.xlabel('Month')
plt.ylabel('Average Air quality Index over last 6 years')
plt.plot(month_list, aqi_list)
plt.show()

In [15]:
import plotly.plotly as py
import plotly.graph_objs as go
month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']
month_list = list(newyork_monthly_aqi['Month'].values.flatten())
aqi_list = list(newyork_monthly_aqi['Avg_AQI'].values.flatten())
# Create a trace
trace = go.Scatter(
    x = month,
    y = aqi_list
)

data = [trace]
layout = dict(title = 'Best time to visit New York',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Avg AQI for past 6 years'),
              )

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='basic-line')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kraghuraman/0 or inside your plot.ly account where it is named 'basic-line'


In [9]:
# Use matplotlib library to plot graph - a simple barchart in this case 
# We encourage you to explore different and more complex types of chart
import matplotlib.pyplot as plt


# First we convert the Dataframe object to Python list (that Matplotlib understands)
from pandas import DataFrame
# this is our 'y-axis' value - obtained by flattening the DataFrame object 
aqis_list = list(avg_air_quality['avg_aqi'].values.flatten())
# This is our x-axis value 
cities = list(avg_air_quality['city'].values.flatten())

# Specify the plot type and formatting 
plt.title('Air quality index with date')
y_pos = np.arange(len(cities))
# Type of graph
plt.bar(y_pos, aqis_list, align='center', alpha=0.6, width=0.4)
# Specify 'ticks' on the x-axis for each item in the list for 'x-axis' values  
plt.xticks(y_pos, cities)
plt.xlabel('City')
plt.ylabel('Average Air quality Index')
plt.show()
# Save this diagram(by default at the directory from where this application is launched)
plt.savefig("figA.png")


<h5> Step 4 : Optional (recommended) : Build Machine Learning models </h5>

<H4>Summary </H4>
<body>
<p>1. Write a brief summary of your findings here. </p>
<p>2. Additionally explore by building a machine learning model using scikit-learn to make predictions and analysis </p>
</body>

<h5> Finally, take a moment to reflect on what we achieved here - we started with a raw dataset, processed them, built models and ran analytics to finally gain insights from the data :) </h5> 